## Шаг 0. Запуск функций и библиотек

In [24]:
import math
import time
import codecs
import requests
import pandas as pd
import networkx as nx
from tqdm import tqdm
from pprint import pprint
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

def gender_counts (access_token, list_of_ids = None, group_id = None):
    
    ''' Функция принимает на вход токен доступа и либо id паблика (group_id), либо список нужных id (list_of_ids). 
    Возвращает подсчитанное количество мужчин и женщин. '''
    
    if list_of_ids == None:
        group = agg_group_member_ids (group_id, access_token)
    else:
        group = list_of_ids
    group_len = len(group)
    a = 0
    b = 300
    all_sexes = []
    while a < group_len:
        time.sleep(1)
        group_string = str(group[a:b])
        group_string = group_string.replace("[", "").replace("]", "").replace(" ", "")
        group_sex = f'https://api.vk.com/method/users.get?user_ids={group_string}&fields=sex&access_token={access_token}&v=5.124'
        response = requests.request("GET", group_sex).json()['response']
        for i in response:
            k = i['sex']
            all_sexes.append(k)
        a = a+300
        b = b+300
    men = Counter(all_sexes)[2]
    women = Counter(all_sexes) [1]
    print (f'Всего в паблике {men} мужчин')
    print (f'Всего в паблике {women} женщин')
    return men, women

def param_counts (access_token, param, list_of_ids = None, group_id = None):
    
    '''
    Функция принимает на вход токен доступа и либо id паблика (group_id), либо список нужных id (list_of_ids). 
    Возвращает полученный от API ответ и подсчитанные значения.
    
    2. param — параметр, который нужно собрать о пользователях. Только один!!!! Иначе будет перегруз оперативной памяти, 
    файл ляжет, а данные будет трудно достать. Допустимые параметры на выбор: sex, bdate, city, country, home_town. 
    Вводятся как строка'''
    
    if list_of_ids == None:
        group = agg_group_member_ids (group_id, access_token)    
    else:
        group = list_of_ids
        
    group_len = len(group)
    a = 0
    b = 300
    all_params = []
    iterel = 1
    while a < group_len:
        # print ('ВНИМАНИЕ, ИТЕРАЦИЯ №', iterel)
        time.sleep(1)
        group_string = str(group[a:b])
        group_string = group_string.replace("[", "").replace("]", "").replace(" ", "")
        group_param = f'https://api.vk.com/method/users.get?user_ids={group_string}&fields={param}&access_token={access_token}&v=5.124'
        response = requests.request("GET", group_param).json()['response']
        # print (response)
        for i in response:
            try:
                k = i[param]
                
                if param == 'bdate':
                    if len (k) > 8:
                        all_params.append (k[-4:]) 
                else:                
                    all_params.append(k)
                # print (all_params)
            except KeyError:
                # print ('Oh No')
                pass
        a = a+300
        b = b+300
        iterel = iterel + 1
    # print (all_params)
    if param in ['sex', 'bdate','home_town']:
        counter_result = Counter(all_params).most_common()
    elif param in ['city', 'country']:
        pure_only = []
        for i in all_params:
            pure_only.append (i['title'])
        counter_result = Counter(pure_only).most_common()
    print (f'Всего в паблике: {counter_result}')
    return response, counter_result

## Шаг 1. Загрузка датасета. Применение функций

In [5]:
df = pd.read_excel ('Id пабликов.xlsx').drop('Unnamed: 0', 1)
df

,id,public
0,233701376,Балканский экспресс
1,366608409,Балканский экспресс
2,595460123,Балканский экспресс
3,135921697,Балканский экспресс
4,40501290,Балканский экспресс
...,...,...
225940,67895283,Она развалилась
225941,76808182,Она развалилась
225942,445644788,Она развалилась
225943,49020922,Она развалилась


In [15]:
ids = sorted (list (df ['id']))
count_values = Counter (ids).most_common()

central_ids = []

for i in count_values:
    if i[1] == 3:
        central_ids.append (i[0])
        
len (central_ids)

6766

In [18]:
central_ids_genders = gender_counts (access_token, list_of_ids = central_ids)
central_ids_genders

Всего в паблике 5843 мужчин
Всего в паблике 918 женщин


(5843, 918)

In [25]:
central_ids_bdates = param_counts (access_token, 'bdate', list_of_ids = central_ids)
central_ids_cities = param_counts (access_token, 'city', list_of_ids = central_ids)
central_ids_countries = param_counts (access_token, 'country', list_of_ids = central_ids)

Всего в паблике: [('1994', 116), ('1990', 99), ('1998', 93), ('1991', 90), ('1999', 90), ('1992', 88), ('1993', 88), ('1996', 86), ('1989', 83), ('1995', 83), ('1988', 81), ('1997', 78), ('1987', 69), ('2000', 61), ('1986', 54), ('1985', 44), ('1984', 39), ('2001', 39), ('1981', 26), ('1982', 25), ('1983', 23), ('2002', 23), ('1980', 20), ('2006', 15), ('1978', 14), ('1917', 12), ('1979', 11), ('1975', 11), ('2003', 11), ('1977', 11), ('1902', 11), ('1976', 7), ('1905', 6), ('2004', 6), ('1964', 4), ('1969', 4), ('1972', 4), ('1973', 4), ('1966', 4), ('1968', 4), ('2005', 4), ('1974', 4), ('1971', 4), ('1920', 3), ('1923', 3), ('1904', 3), ('1961', 3), ('1916', 3), ('1960', 3), ('1945', 3), ('1957', 3), ('1970', 3), ('1965', 3), ('1929', 2), ('1931', 2), ('1914', 2), ('1951', 2), ('1934', 2), ('1912', 2), ('1933', 2), ('1906', 2), ('1958', 2), ('1910', 2), ('1956', 1), ('1911', 1), ('1922', 1), ('1926', 1), ('1953', 1), ('1946', 1), ('1930', 1), ('1959', 1), ('1935', 1), ('1937', 1), (

In [39]:
def all_aggregated (aggregated):
    
    ''' Функция считает, сколько всего id мы успешно (с результатом) обработали на предыдущем этапе, 
    и возвращает полученное число.  '''
    
    all_count = 0

    for i in aggregated [1]:
        all_count = all_count + i[1]

    print ('Всего собрано: ', all_count)
    
    return all_count

In [40]:
bdays_all = all_aggregated (central_ids_bdates)
cities_all = all_aggregated (central_ids_cities)
countries_all = all_aggregated (central_ids_countries)

Всего собрано:  1712
Всего собрано:  3824
Всего собрано:  4611


In [42]:
bdates_df = pd.DataFrame(central_ids_bdates [1], columns = ['year', 'count'])

bdates_df ['percent'] = bdates_df['count'] * 100 / bdays_all

bdates_df

,year,count,percent
0,1994,116,6.775701
1,1990,99,5.782710
2,1998,93,5.432243
3,1991,90,5.257009
4,1999,90,5.257009
...,...,...,...
71,1935,1,0.058411
72,1937,1,0.058411
73,1967,1,0.058411
74,1949,1,0.058411


In [44]:
cities_df = pd.DataFrame(central_ids_cities [1], columns = ['city', 'count'])

cities_df ['percent'] = cities_df['count'] * 100 / cities_all

cities_df

,city,count,percent
0,Москва,961,25.130753
1,Санкт-Петербург,481,12.578452
2,Киев,139,3.634937
3,Минск,93,2.432008
4,Екатеринбург,71,1.856695
...,...,...,...
667,Varna,1,0.026151
668,Северск,1,0.026151
669,Душанбе,1,0.026151
670,Brugge,1,0.026151


In [45]:
countries_df = pd.DataFrame(central_ids_countries [1], columns = ['country', 'count'])

countries_df ['percent'] = countries_df['count'] * 100 / countries_all

countries_df

,country,count,percent
0,Россия,3376,73.216222
1,Украина,467,10.127955
2,Беларусь,190,4.120581
3,Казахстан,61,1.322923
4,США,42,0.910865
...,...,...,...
127,Венесуэла,1,0.021687
128,Монако,1,0.021687
129,Мали,1,0.021687
130,Северная Македония,1,0.021687


## Шаг 2. Анализ и выводы

## Пол

In [89]:
all_genders = central_ids_genders [0] + central_ids_genders [1]
male_percent = central_ids_genders [0] * 100 / all_genders
female_percent = central_ids_genders [1] * 100 / all_genders
print (f'Мужчин {round(male_percent)}%, а женщин {round (female_percent)}%')

Мужчин 86%, а женщин 14%


Согласно данным [Mediascope](https://webindex.mediascope.net/report?byDevice=3&byGeo=2&byMonth=202010&id=16571), в среднем в аудитории ВКонтакте 54% женщин и 46% мужчин.

В исследуемом ядре очевидно превалируют мужчины (на 32% больше мужчин, чем в среднем по ВКонтакте), в то время как женщин подобный контент почти не интересует (на 40% меньше, чем в среднем по ВКонтакте). 

## Возраст

Так как при попытке собрать сразу несколько параметров, оперативная память нашего компьютера перегружена, и Jupyter перестаёт отвечать, нам нужно смотреть на ситуацию окольными путями.

Поэтому мы сравниваем процент подписчиков определённой возрастной группы с данными Медиаскопа: как с данными по всей аудитории (Monthly Reach, October 2020, Desktop+Mobile), так и с данными только по мужчинам, так как 86% нашей ядра составляют мужчины. 

In [90]:
# Возраст по нашему датасету

age_12_24 = 0
age_25_34 = 0
age_35_44 = 0
age_45_54 = 0
age_mr_55 = 0

for i in bdates_df.iterrows():
    
    if int (i[1]['year']) in range(1996, 2009):
        age_12_24 = age_12_24 + i[1]['percent']
    
    if int (i[1]['year']) in range(1986, 1996):
        age_25_34 = age_25_34 + i[1]['percent']
    
    if int (i[1]['year']) in range(1976, 1986):
        age_35_44 = age_35_44 + i[1]['percent']
    
    if int (i[1]['year']) in range(1966, 1976):
        age_45_54 = age_45_54 + i[1]['percent']

    if int (i[1]['year']) < 1966:
        age_mr_55 = age_mr_55 + i[1]['percent']  

title = ['12-24', '25-34', '35-44', '45-54', '55+']
ages = [age_12_24, age_25_34, age_35_44, age_45_54, age_mr_55]

# ПРОЦЕНТ ВОЗРАСТНЫХ ГРУПП ПО ДАННЫМ МЕДИАСКОПА
# https://webindex.mediascope.net/report/sex-and-age?byGeo=2&byDevice=3&byMonth=202010&id=16571
# Предобработка в Excel: высчитывание процентов от общего числа по возрастным группам

mediascope_ages_all = [21.82578885, 25.40598667, 21.56316995, 15.39271909, 15.81233544]
mediascope_ages_men_only = [24.40506842, 27.02410682, 21.81728345, 14.48760727, 12.26593405]


age_df = pd.DataFrame ([title, ages, mediascope_ages, mediascope_ages_men_only]).T
age_df.columns = ['age', '% our data', '% mediascope all', '% mediascope men only']

age_df ['our - all'] = age_df['% our data'] - age_df['% mediascope all']
age_df ['our - men'] = age_df['% our data'] - age_df['% mediascope men only']

age_df

,age,% our data,% mediascope all,% mediascope men only,our - all,our - men
0,12-24,29.5561,21.8258,24.4051,7.73029,5.15101
1,25-34,49.7079,25.406,27.0241,24.302,22.6838
2,35-44,12.8505,21.5632,21.8173,-8.7127,-8.96682
3,45-54,2.51168,15.3927,14.4876,-12.881,-11.9759
4,55+,5.37383,15.8123,12.2659,-10.4385,-6.8921


Таким образом, мы видим, что в ядре подписчиков преобладает аудитория 25-34. В сравнении с общей аудиторией ВКонтакте, показатель 50% подписчиков больше средних значений на 22-24%. Также больше средних значений аудитория 12-24. Посмотрим на неё поближе.

Проведём рубеж в 15-16 лет, так как в этом возрасте человек обычно заканчивает школу и поступает в университет, в связи с чем его интересы и медиа-запросы меняются; а также в 19-20 лет, когда люди обычно заканчивают бакалавриат.

In [91]:
age_12_15 = 0
age_16_19 = 0
age_20_24 = 0

for i in bdates_df.iterrows():
    
    if int (i[1]['year']) in range(2005, 2009):
        age_12_15 = age_12_15 + i[1]['percent']
    
    if int (i[1]['year']) in range(2001, 2005):
        age_16_19 = age_16_19 + i[1]['percent']
        
    if int (i[1]['year']) in range(1996, 2001):
        age_20_24 = age_20_24 + i[1]['percent']
    
print (age_12_15, age_16_19, age_20_24)

1.1098130841121494 4.614485981308412 23.83177570093458


Получается, что наибольший интерес к контенту группы всё равно проявляют совершеннолетние молодые люди от 20 до 34 лет.

## Страны

Мы хотим визуализировать полученные результаты в программе Flourish, чтобы посмотреть на географическое распределение ядра наших пабликов. Однако данные, введённые пользователями ВКонтакте на русском языке, нужно перевести на английский — причём именно в том варианте, который предзадан во Flourish. Для этого выполним следующую ячейку.

Кроме того, как выяснилось, многие пользователи указывают как страну происхождения островные или непризнанные территории. Комментарии по таким территориям ниже.

In [48]:
english_translations = {}

for i in countries_df ['country']:
    write_translation = input (f'Введите перевод для {i}: ')
    english_translations [i] = write_translation

Введите перевод для Россия: Russian Federation
Введите перевод для Украина: Ukraine
Введите перевод для Беларусь: Republic of Belarus
Введите перевод для Казахстан: Republic of Kazakhstan
Введите перевод для США: United States of America
Введите перевод для Германия: Federal Republic of Germany
Введите перевод для Польша: Republic of Poland
Введите перевод для Израиль: State of Israel
Введите перевод для Чехия: Czech Republic
Введите перевод для Сербия: Republic of Serbia
Введите перевод для Великобритания: United Kingdom of Great Britain and Northern Ireland
Введите перевод для Эстония: Republic of Estonia
Введите перевод для Австрия: Republic of Austria
Введите перевод для Финляндия: Republic of Finland
Введите перевод для Япония: Japan
Введите перевод для Азербайджан: Republic of Azerbaijan
Введите перевод для Литва: Republic of Lithuania
Введите перевод для Нидерланды: Kingdom of the Netherlands
Введите перевод для Франция: French Republic
Введите перевод для Канада: Canada
Введите

Ангилья — самоуправляемая британская заморская территория в восточной части Карибского моря.

Бона́йре, Синт-Эста́тиус и Са́ба, также применяется название Кари́бские Нидерла́нды — три особые общины Нидерландов в Карибском море, в архипелаге Малых Антильских островов в Вест-Индии. Также, как и Кюрасао, находятся в составе Королевства Нидерландов, поэтому в датасет заносятся как Curaçao. В QGis и то, и другое — Netherlands Antilles.

Гиблартар — британская заморская территория на южной оконечности Пиренейского полуострова.

Монтсерра́т — заморская территория Великобритании, находящаяся на одноимённом острове, входящем в архипелаг Малые Антильские острова. 

Ниуэ — остров и одноимённое самоуправляемое государственное образование в свободной ассоциации с Новой Зеландией

Остров Кука — архипелаг и одноимённое самоуправляемое государственное образование в свободной ассоциации с Новой Зеландией.

Остров Мэн — коронное владение Британской короны. Расположен в Ирландском море на примерно одинаковом расстоянии от Англии, Ирландии, Шотландии и Уэльса.

Острова Питкерн — единственная заморская территория Великобритании в Тихом океане. Включает в себя пять островов, один из которых обитаем.

Острова Святой Елены, Вознесения и Триста́н-да-Ку́нья — заморская территория Великобритании, расположенная в Южной Атлантике к западу от африканского побережья.

Тайвань — к сожалению, Flourish не даёт выделить Тайвань как отдельное от Китая государство, поэтому здесь мы причисляем его к КНР.

Токелау — зависимая территория Новой Зеландии.

Уоллис и Футуна — острова в южной части Тихого океана, на расстоянии около 2⁄3 пути между Гавайями и Новой Зеландией.

Шпицберген и Ян-Майен — общее название принадлежащих Норвегии архипелага Шпицберген и острова Ян-Майен.

In [53]:
# Добавляем переводы в датасет

translation_column_data = []

for i in countries_df ['country']:
    if i in english_translations.keys():
        translation_column_data.append (english_translations[i])
        
countries_df ['translation'] = translation_column_data

countries_df

,country,count,percent,translation
0,Россия,3376,73.216222,Russian Federation
1,Украина,467,10.127955,Ukraine
2,Беларусь,190,4.120581,Republic of Belarus
3,Казахстан,61,1.322923,Republic of Kazakhstan
4,США,42,0.910865,United States of America
...,...,...,...,...
127,Венесуэла,1,0.021687,Bolivarian Republic of Venezuela
128,Монако,1,0.021687,Principality of Monaco
129,Мали,1,0.021687,Republic of Mali
130,Северная Македония,1,0.021687,Former Yugoslav Republic of Macedonia


In [54]:
countries_df.to_excel ('Страны ядра.xlsx')

В итоге мы визуализировали не в Flourish, а в QGis. Хотя нам стоило бы выбрать не заливку, а кружочки для визуализации, так как данные не нормированы на площадь или на население, на этой визуализации мы хотели скорее выявить регионы, в которых возникает интерес к контенту паблика. И с этой задачей наша визуализация справляется.

Получается, что регион — это Россия, Восточная и Центральная Европа, Казахстан и США.

![img](https://sun9-42.userapi.com/impg/d_zdGMKVHbJ4RIKWqpIggDqGgu3l0Sv1VvJP7A/7hXJt8qcQkE.jpg?size=2560x1810&quality=96&proxy=1&sign=d977ba93a645988fca52c2cfb59b6e2d&type=album)

Мария Казакова (undine_su_menulio), 2020 год.